# Blog with images

We use this example of creating a web search-based blog with illustrations to show how to 

* instantiate one agent each from Langchain, CrewAI, and LlamaIndex
* give agents as tools to other agents
* create tasks and link them using the >> operator

In [1]:
from pathlib import Path
import os
import sys
import platform

from dotenv import load_dotenv
from langchain_community.tools import DuckDuckGoSearchRun

import kuzu

WORKING_DIR = Path(os.path.realpath("."))

try:
    from motleycrew import MotleyCrew
except ImportError:
    # if we are running this from source
    motleycrew_location = os.path.realpath(WORKING_DIR / "..")
    sys.path.append(motleycrew_location)

from motleycrew.storage import MotleyKuzuGraphStore
from motleycrew import MotleyCrew
from motleycrew.agents.crewai import CrewAIMotleyAgent
from motleycrew.agents.langchain.react import ReActMotleyAgent
from motleycrew.agents.llama_index import ReActLlamaIndexMotleyAgent
from motleycrew.tools.image.dall_e import DallEImageGeneratorTool
from motleycrew.common import configure_logging
from motleycrew.tasks import SimpleTask

configure_logging(verbose=True)
load_dotenv()

True

First, we create a fairly conventional agent that has a search tool. 

In [2]:
search_tool = DuckDuckGoSearchRun()

researcher = CrewAIMotleyAgent(
    role="Senior Research Analyst",
    goal="Uncover cutting-edge developments in AI and data science, doing web search if necessary",
    backstory="""You work at a leading tech think tank.
Your expertise lies in identifying emerging trends.
You have a knack for dissecting complex data and presenting actionable insights.""",
    verbose=True,
    tools=[search_tool],
)

Now the fun starts happening: we pass that agent to another agent as a tool. That's how delegation works in motleycrew: you just pass agents as tools to other agents; the API is automatically converted under the hood.

In [3]:
# You can give agents as tools to other agents
writer = ReActMotleyAgent(
    name="AI writer agent",
    prompt_prefix="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
              Identify key trends, breakthrough technologies, and potential industry impacts.
              Your final answer MUST be a full analysis report""",
    tools=[researcher],
    verbose=True,
)


Let's create a first task, writing a blog post, and give it to the writer agent. We'll need to instantiate a crew first as a crew orchestrates the tasks.

In [4]:
# You always need a crew to orchestrate the agents
crew = MotleyCrew()

blog_post_task = SimpleTask(
    crew=crew,
    name="produce blog post on AI advancements",
    description="""Using the insights provided by a thorough web search, develop an engaging blog
post that highlights the most significant AI advancements.
Your post should be informative yet accessible, catering to a tech-savvy audience.
Make it sound cool, avoid complex words so it doesn't sound like AI.
Create a blog post of at least 4 paragraphs, in markdown format.""",
    agent=writer,
)

2024-05-26 13:23:02,189 - INFO - No db_path provided, creating temporary directory for database
2024-05-26 13:23:02,191 - INFO - Using Kuzu graph store with path: C:\Users\Egor\AppData\Local\Temp\tmpmdjgnviv\kuzu_db
2024-05-26 13:23:02,222 - INFO - Node table MotleyGraphNode does not exist in the database, creating
2024-05-26 13:23:02,245 - INFO - Relation table dummy from MotleyGraphNode to MotleyGraphNode does not exist in the database, creating
2024-05-26 13:23:02,264 - INFO - Node table TaskNode does not exist in the database, creating
2024-05-26 13:23:02,281 - INFO - Property name not present in table for label TaskNode, creating
2024-05-26 13:23:02,299 - INFO - Property done not present in table for label TaskNode, creating
2024-05-26 13:23:02,316 - INFO - Inserting new node with label TaskNode: name='produce blog post on AI advancements' done=False
2024-05-26 13:23:02,345 - INFO - Node created OK
2024-05-26 13:23:02,348 - INFO - Relation table task_is_upstream from TaskNode to T

Now let's create a second task, to generate and insert into the blog some illustrations.

In [9]:
# Illustrator
image_generator_tool = DallEImageGeneratorTool(os.path.realpath("./images"),
                                               model="dall-e-3",
                                              refine_prompt_with_llm=True,
                                              size="1024x1024",
                                              style="vivid")

illustrator = ReActLlamaIndexMotleyAgent(
    name="Illustrator",
    description="Create beautiful and insightful illustrations for a blog post",
    tools=[image_generator_tool],
)

illustration_task = SimpleTask(
    crew=crew,
    name="create an illustration for the blog post",
    description="""Create beautiful and insightful illustrations to accompany the blog post on AI advancements.
    The blog post will be provided to you in markdown format.
    Make sure to use the illustration tool provided to you, once per illustration, and embed the URL provided by
    the tool into the blog post.
    Create between 2 and 3 illustrations, neither more nor less
    Only return the full text of the blog post with embedded URLs, no backticks, no other text.
    Only use slashes as path separators, never backslashes""",
    agent=illustrator,
)

2024-05-26 13:25:49,037 - INFO - Inserting new node with label TaskNode: name='create an illustration for the blog post' done=False
2024-05-26 13:25:49,064 - INFO - Node created OK


Now let's connect the two tasks, and run them!

In [10]:
# Make sure the illustration task runs only once the blog post task is complete, and gets its input
blog_post_task >> illustration_task

# Get your crew to work!
result = crew.run()
final_result = illustration_task.output

2024-05-26 13:25:50,262 - INFO - Creating relation task_is_upstream from TaskNode:0 to TaskNode:2
2024-05-26 13:25:50,290 - INFO - Relation created OK
2024-05-26 13:25:50,292 - WARNING - Multithreading is not implemented yet, will run in single thread
2024-05-26 13:25:50,306 - INFO - Available tasks: [SimpleTask(name=create an illustration for the blog post, done=False)]
2024-05-26 13:25:50,306 - INFO - Processing task: SimpleTask(name=create an illustration for the blog post, done=False)
2024-05-26 13:25:50,348 - INFO - Got a matching unit for task SimpleTask(name=create an illustration for the blog post, done=False)
2024-05-26 13:25:50,349 - INFO - Processing task: TaskUnit(status=pending)
2024-05-26 13:25:50,349 - INFO - Assigned unit TaskUnit(status=pending) to agent Agent(name=Illustrator), dispatching
2024-05-26 13:25:50,350 - INFO - Node TaskUnit(status=running) does not exist, creating
2024-05-26 13:25:50,352 - INFO - Inserting new node with label SimpleTaskUnit: TaskUnit(statu

Let's display the result (swapping absolute for relative paths, as Jupyter seems to struggle with the former)

In [11]:
from IPython.display import Markdown, display
here = os.path.realpath(".").replace(os.sep,"/")
display(Markdown(illustration_task.output.replace(here,".")))

# Exploring the Frontier: The Latest Breakthroughs in AI

Welcome to the cutting edge of technology! Today, we're diving into some of the most exciting advancements in artificial intelligence that are setting the stage for a future where the possibilities seem limitless.

## Understanding AI Like Never Before
One of the most significant strides in AI recently is the push towards making these systems more understandable. Thanks to initiatives by organizations like DARPA, there's a strong focus on developing AI technologies that are not only powerful but also transparent and trustworthy. This move is crucial as it helps bridge the gap between AI capabilities and human trust, ensuring that AI systems can be reliable partners in various applications.
![Understanding AI Like Never Before](./images/4b31023c.png)

## The Rise of Generative AI
Imagine an AI that can create content that feels incredibly human-like, whether it's writing a story, composing music, or generating realistic images. That's what Generative AI is all about, and it's rapidly moving from lab experiments to real-world applications. This technology is not just about mimicking human creativity; it's about augmenting our capabilities and providing tools that can inspire and accelerate innovation in countless domains.
![The Rise of Generative AI](./images/5ac868dc.png)

## Gemini Ultra: A New Benchmark in AI Excellence
The development of Gemini Ultra marks a monumental achievement in the field of large language models (LLMs). This model has not only surpassed human experts in multiple benchmarks but has also set new records, achieving a score of 90.04% on the MMLU and 59.4% on the MMMU. Gemini Ultra's performance highlights the incredible potential of AI to handle complex, nuanced tasks that were once thought to be the exclusive domain of human intelligence.

## Multimodality: The Next Dimension of AI
As AI continues to evolve, the ability to process and integrate multiple types of data — text, images, sound, and more — is becoming increasingly important. This multimodality allows AI systems to have a deeper understanding of the world and enhances their applicability across different sectors, from healthcare to entertainment. By harnessing the power of multimodal AI, we're paving the way for more versatile and capable systems that can better understand and interact with their environment.
![Multimodality: The Next Dimension of AI](./images/28bbbdad.png)

As we look to the future, these advancements in AI are not just technological milestones; they're gateways to new possibilities that will redefine what's possible. Stay tuned, because the journey into the future of AI is just getting started!